## Reinforcement Learning

Im Gegensatz zu klassischen ML-Methoden gibt es bei RL kein vorgegebenes Datenset

Das Datenset wird während des Prozesses erzeugt, und darauf wird der Algorithmus trainiert

Prozess:
- Was soll gelernt werden
    - Reale Prozesse (Roboter)
    - Softwareprozesse (Spiele)
 
Umgebung:
- Konkrete Implementation von dem Prozess
- Umgebung wird ausgeführt und generiert die Daten

Agent:
- "Der Akteur", welcher den Prozess/Umgebung ausführt
- Lernt, generiert das Modell und kann dieses auch verwenden

Pakete:
- Gymnasium (https://github.com/Farama-Foundation/Gymnasium)
- Stable-Baselines 3

In [1]:
import gymnasium as gym

### Vorgegebene Umgebungen

Von OpenAI gibt es Demoumgebungen, welche zum testen verwendet werden können

Eine davon ist CartPole-v1

Es geht darum ein Kart zu bewegen, um eine darauf angebrachte Stange gerade zu halten

https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [11]:
env = gym.make("CartPole-v1", render_mode="human")

### Inhalte einer Umgebung

Daten:
- Action Space: Was kann der Agent tun? (hier Links bewegen, rechts bewegen)
- Observation Space: Die Form der Daten, welche während des Prozesses erzeugt werdn
    - Wird vom ML-Algorithmus konsumiert
 
Funktionen:
- step: Führt den Prozess um einen Schritt aus
    - Bei jedem Step kommt eine Belohnung heraus, welche den Erfolg des Agenten beschreibt
- reset: Setzt die Umgebung auf den Anfang zurück, wird für ML verwendet
- render: Zeichne die Umgebung
- close: Beende die Umgebung, Aufräumen (Fenster schließen, ...)

### Agent definieren

In [12]:
d = 5
for x in range(d):
    start = env.reset()  # [ 0.0492511 ,  0.04087935, -0.04937328, -0.00469788] (Startpos, StartV, StartPolePos, StartPoleV)
    score = 0
    done = False
    
    while not done:
        env.render()  # Umgebung zeichnen
        action = env.action_space.sample()  # Generiere eine Random Action
        newState, reward, done, term, info = env.step(action)  # Bei einer Action kommen 5 Werte zurück: Neuer State, Belohnung, Fertig?, Mach Ende?, Info
        score += reward
    print(f"Durchgang: {x}, Score: {score}")
env.close()

Durchgang: 0, Score: 10.0
Durchgang: 1, Score: 20.0
Durchgang: 2, Score: 9.0
Durchgang: 3, Score: 28.0
Durchgang: 4, Score: 26.0


### RL Modell trainieren

Trainingsalgorithmus:
- PPO: Proximal Policy Optimization
- https://stable-baselines.readthedocs.io/en/master/guide/algos.html

Policy:
- Bestimmt, wie das unterliegende Modell aufgebaut ist
- https://stable-baselines.readthedocs.io/en/master/modules/policies.html

DummyVecEnv:
- Simulation mehrerer Durchgänge gleichzeitig
- Nützlich um Zeit zu sparen

In [14]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

In [17]:
env = gym.make("CartPole-v1")
env = DummyVecEnv([lambda: env])
model = PPO("MlpPolicy", env, verbose=1)

Using cpu device


In [18]:
model.learn(total_timesteps=200_000)

-----------------------------
| time/              |      |
|    fps             | 687  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 335         |
|    iterations           | 2           |
|    time_elapsed         | 12          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008432631 |
|    clip_fraction        | 0.109       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00201    |
|    learning_rate        | 0.0003      |
|    loss                 | 4.56        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0167     |
|    value_loss           | 45.1        |
-----------------------------------------
----------------------------------

KeyboardInterrupt: 

In [19]:
model.save("Models/CartPole")

In [41]:
env = gym.make("CartPole-v1")

In [42]:
model = PPO.load("Models/CartPole", env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [43]:
d = 5
for x in range(d):
    state = env.reset()  # [ 0.0492511 ,  0.04087935, -0.04937328, -0.00469788] (Startpos, StartV, StartPolePos, StartPoleV)
    score = 0
    done = False
    
    while not done:
        # env.render()  # Umgebung zeichnen
        if len(state) == 2:
            action, _ = model.predict(state[0])  # Generiere eine Random Action
        else:
            action, _ = model.predict(state)
        state, reward, done, term, info = env.step(action)  # Bei einer Action kommen 5 Werte zurück: Neuer State, Belohnung, Fertig?, Mach Ende?, Info
        score += reward
    print(f"Durchgang: {x}, Score: {score}")
env.close()

Durchgang: 0, Score: 1290.0
Durchgang: 1, Score: 990.0
Durchgang: 2, Score: 1112.0
Durchgang: 3, Score: 1122.0
Durchgang: 4, Score: 3187.0
